# Figures for paper

In [2]:
import pandas as pd
import numpy as np
import math
from math import log10
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import alphashape

from matplotlib.ticker import FormatStrFormatter
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.patches import Polygon

from scipy.interpolate import CubicSpline
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit
from sklearn import linear_model




ModuleNotFoundError: No module named 'alphashape'

Import data

In [ ]:
data=pd.read_csv('combined models.csv')
# correction of n_values in dd survey
data.loc[data['array']=='dd','n_value']=(data['m']-data['b'])/data['b']
data['n_value']=data['n_value'].round(0)
data.to_csv('combined models_test.csv')
print(len(data))
#data=data.sort_values('doi')
data
print(data.cw.unique())
resistivity_contrasts_log=[-2,-5/3,-4/3,-1,-2/3,-1/3,0,1/3,2/3,1,4/3,5/3,2]
resistivity_contrasts=[round(10**rc,3) for rc in resistivity_contrasts_log]
print(resistivity_contrasts)
#remove duplicate run from data
data=data.drop(list(range(57531,57984,1)),axis=0).reset_index(drop=True)

Data check that everything is in the data file

In [ ]:
data_count = data.groupby(['cw', 'cot']).size().reset_index(name='counts')
print(f'Max count: {data_count["counts"].max()}')
print(f'Min count: {data_count["counts"].min()}')
ax=plt.scatter(x=data_count['cw'],y=data_count['cot'],s=100, c=data_count['counts'],cmap='turbo',vmin=data_count["counts"].min()-1,vmax=data_count["counts"].max()+1)
ax = plt.gca()
plt.xscale('log')
plt.ylabel("Slope angle")
plt.xlabel("Crest width normlised to 1m hight")
plt.colorbar().ax.set_ylabel('Counts', rotation=270, labelpad=15)

In [ ]:
# calculate different E value
#data_test=data.copy()
for k in resistivity_contrasts:
    print(k)
    #data[f'E_{k}']=(data[f'3D_{k}']-data[f'2D_{k}'])/(data[f'2D_{k}'])*100

    data[f'E_{k}']=data[f'3D_{k}']/data[f'2D_{k}']
    
    #sanity check
    #data_test[f'E3D_{k}']=(data_test[f'3D_{k}']-data_test[f'2D_{k}'])/(data_test[f'3D_{k}'])*100
    #logE_res=log10(2D/3D)
    #data_test[f'logE3D_{k}']=np.log10(data_test[f'2D_{k}']/data_test[f'3D_{k}'])
    #E2D_res
    #data_test[f'E2D_{k}']=(data_test[f'3D_{k}']-data_test[f'2D_{k}'])/(data_test[f'2D_{k}'])*100
    #logE2d_res
    #data_test[f'logE2D_{k}']=np.log10(data_test[f'2D_{k}']/data_test[f'3D_{k}'])

# Checking modelling errors

In [ ]:
fig,ax=plt.subplots(figsize=(6, 4))
font = {'size'   : 12}
mpl.rc('font', **font)

cot=3
cw=5.27#5.27
res=100
data_filter=(data['cot']==cot) & (data['cw']==cw)
filtered_data=data[data_filter]
temp=filtered_data[filtered_data['E_100']==1][['a', 'b', 'm', 'n','2D_100', '3D_100','E_100']]
temp
#E_twod_norm=(filtered_data[f'3D_{res}']-filtered_data[f'2D_{res}'])*100/filtered_data[f'3D_{res}']
#filtered_data.loc[:,f'E_twod_{res}']=E_twod_norm

for array in [ 'mg','sh','w','dd']:#filtered_data['array'].unique():
    m_type={'sh':'+','w':'+','dd':'.','mg':'x'}
    data_by_ar=filtered_data[filtered_data['array']==array]
    if array=='dd':
        n_value=(data_by_ar['m']-data_by_ar['b'])/data_by_ar['b']
        data_by_ar.loc[:,'n_value']=n_value
        print(f'dd {len(data_by_ar)}')
        colour=ax.scatter(x=data_by_ar[f'E_{res}'],y=data_by_ar['doi'],s=10,marker=m_type[array],c=data_by_ar['n_value'],cmap='turbo',label='Dipole-Dipole')
    
    elif array=='mg':
        ax.scatter(x=data_by_ar[f'E_{res}'],y=data_by_ar['doi'],s=20,marker=m_type[array],c='0.8',label='Multiple Gradient ')#,c=data_by_ar['n_value'],cmap='jet')

    else:
        ax.scatter(x=data_by_ar[f'E_{res}'],y=data_by_ar['doi'],s=20,marker=m_type[array],c=data_by_ar['n_value'],cmap='turbo',label='Wenner-Schlumberger')
        print(f'w {len(data_by_ar)}')
        
ax.set_yscale('log')
ax.set_xscale('log')
ax.invert_yaxis()
ax.grid(which='major')
ax.grid(which='minor',lw=0.2)
ax.set(xlabel="Topographic Effect - E (Ω/Ω)",ylabel="Effective Depth (m/m)",xscale='log')

#plt.title(f'cot(θ) = {cot}, crest width = {cw}')
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(),prop={'size': 10})

ax.yaxis.set_major_formatter(ticker.ScalarFormatter())

ax.tick_params(which='both', length=2)
ax.set_xticks([1])
ax.set_xticklabels(['1'])
ax.xaxis.set_minor_locator(ticker.FixedLocator(locs=[1.0005,1.0010,1.0015,1.0020,1.0025]))
ax.xaxis.set_minor_formatter(ticker.FixedFormatter(['1.0005','','1.0015','','1.0025']))


fig.colorbar(colour).ax.set_ylabel('$\it{n}$', rotation=270, labelpad=15)
plt.savefig(f'Figures/Meshing error plot.svg',format='svg',bbox_inches = 'tight')

# Figure 5: Comparing a & n values and array types

In [ ]:
fig,ax=plt.subplots(figsize=(6, 4))
font = {'size'   : 12}
mpl.rc('font', **font)

cot=3
cw=2.1
res=0.1
data_filter=(data['cot']==cot) & (data['cw']==cw)
filtered_data=data[data_filter]

#E_twod_norm=(filtered_data[f'3D_{res}']-filtered_data[f'2D_{res}'])*100/filtered_data[f'3D_{res}']
#filtered_data.loc[:,f'E_twod_{res}']=E_twod_norm

for array in [ 'mg','sh','w','dd']:#filtered_data['array'].unique():
    m_type={'sh':'+','w':'+','dd':'.','mg':'x'}
    data_by_ar=filtered_data[filtered_data['array']==array]
    if array=='dd':
        n_value=(data_by_ar['m']-data_by_ar['b'])/data_by_ar['b']
        data_by_ar.loc[:,'n_value']=n_value
        print(f'dd {len(data_by_ar)}')
        colour=ax.scatter(x=data_by_ar[f'E_{res}'],y=data_by_ar['doi'],s=10,marker=m_type[array],c=data_by_ar['n_value'],cmap='turbo',label='Dipole-Dipole')
    
    elif array=='mg':
        ax.scatter(x=data_by_ar[f'E_{res}'],y=data_by_ar['doi'],s=20,marker=m_type[array],c='0.8',label='Multiple Gradient ')#,c=data_by_ar['n_value'],cmap='jet')

    else:
        ax.scatter(x=data_by_ar[f'E_{res}'],y=data_by_ar['doi'],s=20,marker=m_type[array],c=data_by_ar['n_value'],cmap='turbo',label='Wenner-Schlumberger')
        print(f'w {len(data_by_ar)}')
        
ax.set_yscale('log')
ax.invert_yaxis()
ax.grid(which='major')
ax.grid(which='minor',lw=0.2)
ax.set(xlabel="Topographic Effect - E (Ω/Ω)",ylabel="Effective Depth (m/m)",xscale='log')

#plt.title(f'cot(θ) = {cot}, crest width = {cw}')
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(),prop={'size': 10})

ax.yaxis.set_major_formatter(ticker.ScalarFormatter())

ax.tick_params(which='both', length=2)
ax.set_xticks([1,2])
ax.set_xticklabels(['1','2'])
ax.xaxis.set_minor_locator(ticker.FixedLocator(locs=[0.9,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9]))
ax.xaxis.set_minor_formatter(ticker.FixedFormatter(['0.9','','1.2','','','1.5','','','','']))


fig.colorbar(colour).ax.set_ylabel('$\it{n}$', rotation=270, labelpad=15)
plt.savefig(f'Figures/Figure 3.svg',format='svg',bbox_inches = 'tight')

# Filter data for n_value=1

In [ ]:
#removed all data where n != 1, this leeves the fundamental dipole-dipole and wenner array.
data_n1=data[data['n_value']==1].reset_index(inplace=False)
data_n1=data_n1.groupby(['index','array','doi']).mean().reset_index()
data_n1=data_n1.sort_values(by=['index','array', 'doi'])


## Figure 6 Topography only 

In [ ]:
# expanded version of figure plotting
# raw data no interpolation
fig, ((A, B), (C, D)) = plt.subplots(2, 2,sharex=True, sharey=True)
fig.set_size_inches(7, 5)
plt.subplots_adjust(wspace=0.08, hspace=0.08)
font = {'size'   : 15}

#A
#Filter data
cot=3
res=1
array='w'
angle_filter = data_n1['cot'] == cot
array_filter = data_n1['array'] == array
filtered=data_n1[angle_filter & array_filter]

#Plot line data
cw=sorted(filtered['cw'].unique())
cw_log=np.log10(cw)
colour = iter(plt.cm.turbo((list(cw_log)-min(cw_log))/(max(cw_log)-min(cw_log))))
for i,c in zip(cw, colour):
    filtered_cw=filtered[filtered['cw']==i]
    x=filtered_cw[f'E_{res}']
    y=filtered_cw['doi']
    spl = CubicSpline(y, x)
    y_new=np.logspace(log10(y.min()),log10(y.max()),num=1001)
    A.plot(spl(y_new),y_new,label=i,color=c)

#Plot data
A_plot=A.scatter(x=filtered[f'E_{res}'],y=filtered['doi'],s=0.1,c=filtered['cw'],cmap='viridis',norm=mpl.colors.LogNorm())
A.set_yscale('log')
A.invert_yaxis()
A.grid(which='major')
A.grid(which='minor',lw=0.2)
A.set(ylabel="Effective depth - E [m]")#, title=f'cot(θ) = {cot}, crest width = {cw}', xlabel="E [%]",
majors = ["0.1","1","10"]
A.yaxis.set_major_locator(ticker.FixedLocator(locs=[0.1,1,10]))
A.yaxis.set_major_formatter(ticker.FixedFormatter(majors))


#B
cot=3
res=1
array='dd'
angle_filter = data_n1['cot'] == cot
array_filter = data_n1['array'] == array
filtered = data_n1[angle_filter & array_filter]

#Plot line data

colour = iter(plt.cm.turbo(np.linspace(0, 1, len(filtered['cw'].unique()))))
cw=sorted(filtered['cw'].unique())
for i,c in zip(cw, colour):
    filtered_cw=filtered[filtered['cw']==i]
    B.plot(filtered_cw[f'E_{res}'],filtered_cw['doi'],label=i,color=c)

B_plot=B.scatter(x=filtered[f'E_{res}'],y=filtered['doi'],s=0.1,c=filtered['cw'],cmap='turbo',norm=mpl.colors.LogNorm())
B.grid(which='major')
B.grid(which='minor',lw=0.2)
#B.set(xlabel="E [%]")#ylabel="Effective depth [m]", title=f'cot(θ) = {cot}, crest width = {cw}')
#fig.colorbar(B_plot,ax=B).ax.set_ylabel('Slope angle', rotation=270, labelpad=15)
B.axes.yaxis.set_ticklabels([])

divider = make_axes_locatable(B)
cax = divider.append_axes('right', size='4%', pad=0.1)
cbar=fig.colorbar(B_plot, cax=cax, orientation='vertical')

#cbar=plt.colorbar(B_plot,ax=B)
majors = ["1"]
minors = ["0.2","0.5","2","3","4","5"]
cbar.ax.yaxis.set_major_locator(ticker.FixedLocator(locs=[1]))
cbar.ax.yaxis.set_major_formatter(ticker.FixedFormatter(majors))
cbar.ax.yaxis.set_minor_locator(ticker.FixedLocator(locs=[0.2,0.5,2,3,4,5]))
cbar.ax.yaxis.set_minor_formatter(ticker.FixedFormatter(minors))
cbar.ax.set_ylabel(f'Crest Width', rotation=270, labelpad=15)

#C
#Filter data
cw=2.1
res=1
array='w'
crest_filter=data_n1['cw']==cw
array_filter = data_n1['array'] == array
filtered = data_n1[crest_filter & array_filter]
colour = iter(plt.cm.turbo(np.linspace(0, 1, len(filtered['cot'].unique()))))

#Plot line data
cot=sorted(filtered['cot'].unique())
for i,c in zip(cot, colour):
    filtered_cw=filtered[filtered['cot']==i]
    C.plot(filtered_cw[f'E_{res}'],filtered_cw['doi'],label=i,color=c)

#Plot point data
C_plot=C.scatter(x=filtered[f'E_{res}'],y=filtered['doi'],s=0.1,c=filtered['cot'],cmap='turbo')
C.grid(which='major')
C.grid(which='minor',lw=0.2)
C.set(xlabel="E [%]",ylabel="Effective depth [m]")#, title=f'cot(θ) = {cot}, crest width = {cw}')
majors = ["0.1","1","10"]
C.yaxis.set_major_locator(ticker.FixedLocator(locs=[0.1,1,10]))
C.yaxis.set_major_formatter(ticker.FixedFormatter(majors))


#D
#Filter data
cw=2.1
res=1
array='dd'
crest_filter=data_n1['cw']==cw
array_filter = data_n1['array'] == array
filtered = data_n1[crest_filter & array_filter]

#Plot line data
colour = iter(plt.cm.turbo(np.linspace(0, 1, len(filtered['cot'].unique()))))
cot=sorted(filtered['cot'].unique())
for i,c in zip(cot, colour):
    filtered_cw=filtered[filtered['cot']==i]
    D.plot(filtered_cw[f'E_{res}'],filtered_cw['doi'],label=i,color=c)

#Plot scatter data
D_plot=D.scatter(x=filtered[f'E_{res}'],y=filtered['doi'],s=0.1,c=filtered['cot'],cmap='turbo')
D.grid(which='major')
D.grid(which='minor',lw=0.2)
D.set(xlabel="E [%]")#ylabel="Effective depth [m]", title=f'cot(θ) = {cot}, crest width = {cw}')

#Plot colour bar
divider = make_axes_locatable(D)
cax = divider.append_axes('right', size='4%', pad=0.1)
cbar=fig.colorbar(D_plot, cax=cax, orientation='vertical')
cbar.ax.set_ylabel(f'Slope angle', rotation=270, labelpad=15)

#Annotations
A.text(-0.07, 1, "A", size=10,transform=A.transAxes, weight='bold')
B.text(-0.07, 1, "B", size=10,transform=B.transAxes, weight='bold')
C.text(-0.07, 1, "C", size=10,transform=C.transAxes, weight='bold')
D.text(-0.07, 1, "D", size=10,transform=D.transAxes, weight='bold')
A.text(0.98,0.02,f'Wenner', ha='right', va='bottom',transform=A.transAxes).set_bbox(dict(facecolor='white', alpha=0.5,linewidth=0,pad=0.1))
B.text(0.98,0.02,f'Dipole-Dipole', ha='right', va='bottom',transform=B.transAxes).set_bbox(dict(facecolor='white', alpha=0.5,linewidth=0,pad=0.1))
C.text(0.98,0.02,f'Wenner', ha='right', va='bottom',transform=C.transAxes).set_bbox(dict(facecolor='white', alpha=0.5,linewidth=0,pad=0.1))
D.text(0.98,0.02,f'Dipole-Dipole', ha='right', va='bottom',transform=D.transAxes).set_bbox(dict(facecolor='white', alpha=0.5,linewidth=0,pad=0.1))

#Save figure
plt.savefig('Figures/Figrue 4 topo errors line.svg',format='svg',transparent=True,pad_inches=0)




In [ ]:
# Same plot as before but interpolation between points for a smoother curves.
fig, ((A, B), (C, D)) = plt.subplots(2, 2,sharex=True, sharey=True,figsize=(7,5))
#fig.set_size_inches(7, 5)
plt.subplots_adjust(wspace=0.08, hspace=0.08)
plt.rcParams.update({'font.size': 10})

fixed_var=['cot','cot','cw','cw'] #filter by parameter
fixed_val=[3,3,2.1,2.1] #  parameter value
variable=['cw','cw','cot','cot'] #variable
res_list=[1,1,1,1] #variable value
array_list=['w','dd','w','dd'] # array type
graphs=['A','B','C','D'] # axis
arrays={'w':'Wenner','dd':'Dipole-Dipole'} #dictionary lookup for labels

for i,ax in enumerate([A,B,C,D]):
    graph=graphs[i]    
    geo_filter=data_n1[f'{fixed_var[i]}']==fixed_val[i]
    array_filter = data_n1['array'] == array_list[i]
    filtered = data_n1[geo_filter & array_filter]
    
    #Plot line data
    var=sorted(filtered[variable[i]].unique())
    var_log=np.log10(var)
    colour = iter(plt.cm.turbo((list(var_log)-min(var_log))/(max(var_log)-min(var_log))))
    for v,c in zip(var, colour):
        filtered_cw=filtered[filtered[variable[i]]==v]
        #filtered_cw=filtered_cw.groupby('doi').mean().reset_index()
        x=filtered_cw[f'E_{res_list[i]}']
        y=filtered_cw['doi']
        spl = CubicSpline(y, x)
        y_new=np.logspace(log10(y.min()),log10(y.max()),num=1001)
        ax.plot(spl(y_new),y_new,label=i,color=c,zorder=1)
    
    #Plot scatter data
    ax_plot=ax.scatter(x=filtered[f'E_{res}'],y=filtered['doi'],s=0.1,c='0',alpha=0.3,zorder=2)
    ax.set_yscale('log')
    #ax.invert_yaxis()
    ax.grid(which='major')
    ax.grid(which='minor',lw=0.2)
    majors = ["0.1","1","10"]
    ax.yaxis.set_major_locator(ticker.FixedLocator(locs=[0.1,1,10]))
    ax.yaxis.set_major_formatter(ticker.FixedFormatter(majors))
    ax.text(-0.07, 1, graphs[i], size=10,transform=ax.transAxes, weight='bold')
    ax.text(0.98,0.02,arrays[f'{array_list[i]}'], ha='right', va='bottom',transform=ax.transAxes).set_bbox(dict(facecolor='white', alpha=0.5,linewidth=0,pad=0.1))
    
    #plot colour bar 
    
    if graphs[i]=='B' or graphs[i]=='D':
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='4%', pad=0.1)
        cMin=filtered[variable[i]].min()
        cMax=filtered[variable[i]].max()
        norm=mpl.colors.LogNorm(vmin=cMin, vmax=cMax)

        if graphs[i]=='B':
            ticks=[0.2,0.5,1,2,5] # crest widths
            cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='turbo'),cax=cax,ticks=ticks,format='%.1f')
            cbar.ax.set_ylabel(f'Crest Width (m/m)', rotation=270, labelpad=15)
        if graphs[i]=='D':
            ticks=[1,1.5,2,2.5,3]
            cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='turbo'),cax=cax,ticks=ticks,format='%.1f')
            cbar.ax.set_ylabel(f'Slope angle (1:x)', rotation=270, labelpad=15)


ax.invert_yaxis()
A.set(ylabel="Effective depth (m/m)")#, title=f'cot(θ) = {cot}, crest width = {cw}'",
C.set(xlabel="Topographic effect - E (Ω/Ω)",ylabel="Effective depth (m/m)",xscale='log')
D.set(xlabel="Topographic effect - E (Ω/Ω)",xscale='log')
ax.tick_params(which='both', length=2)

ax.xaxis.set_minor_locator(ticker.FixedLocator(locs=[1.1,1.2,1.3]))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
ax.xaxis.set_minor_formatter(ticker.ScalarFormatter())


#Save figure
plt.savefig('Figures/Figrue 4 topo errors line interp.svg',format='svg',transparent=True,bbox_inches='tight',pad_inches=0.1)

# Supplementary Figure S1

In [ ]:
# Same plot as before but interpolation between points for a smoother curves.
fig, ((A, B), (C, D)) = plt.subplots(2, 2,sharex='col', sharey=True,figsize=(7,5))
#fig.set_size_inches(7, 5)
plt.subplots_adjust(wspace=0.08, hspace=0.08)
plt.rcParams.update({'font.size': 10})

fixed_var=['cot','cot','cw','cw'] #filter by parameter
fixed_val=[2,2,0.42,1.05] #  parameter value
variable=['cw','cw','cot','cot'] #variable
res_list=[100,0.01,100,0.01] #variable value
array_list=['dd','dd','dd','dd'] # array type
graphs=['A','B','C','D'] # axis
arrays={'w':'Wenner','dd':'Dipole-Dipole'} #dictionary lookup for labels

for i,ax in enumerate([A,B,C,D]):
    graph=graphs[i]    
    geo_filter=data_n1[f'{fixed_var[i]}']==fixed_val[i]
    array_filter = data_n1['array'] == array_list[i]
    filtered = data_n1[geo_filter & array_filter]
    
    #Plot line data
    var=sorted(filtered[variable[i]].unique())
    var_log=np.log10(var)
    colour = iter(plt.cm.turbo((list(var_log)-min(var_log))/(max(var_log)-min(var_log))))
    for v,c in zip(var, colour):
        filtered_cw=filtered[filtered[variable[i]]==v]
        #filtered_cw=filtered_cw.groupby('doi').mean().reset_index()
        x=filtered_cw[f'E_{res_list[i]}']
        y=filtered_cw['doi']
        spl = CubicSpline(y, x)
        y_new=np.logspace(log10(y.min()),log10(y.max()),num=1001)
        ax.plot(spl(y_new),y_new,label=i,color=c,zorder=1)
        ax.scatter(x,y,s=0.1,c='0',alpha=0.3,zorder=2)
    
    #Plot scatter data
    #ax_plot=ax.scatter(x=filtered[f'E_{res}'],y=filtered['doi'],s=0.1,c='0',alpha=0.3,zorder=2)
    ax.set_yscale('log')
    #ax.invert_yaxis()
    ax.grid(which='major')
    ax.grid(which='minor',lw=0.2)
    majors = ["0.1","1","10"]
    ax.yaxis.set_major_locator(ticker.FixedLocator(locs=[0.1,1,10]))
    ax.yaxis.set_major_formatter(ticker.FixedFormatter(majors))
    ax.text(-0.07, 1, graphs[i], size=10,transform=ax.transAxes, weight='bold')
    
    #plot colour bar 
    
    if graphs[i]=='B' or graphs[i]=='D':
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='4%', pad=0.1)
        cMin=filtered[variable[i]].min()
        cMax=filtered[variable[i]].max()
        norm=mpl.colors.LogNorm(vmin=cMin, vmax=cMax)

        if graphs[i]=='B':
            ticks=[0.2,0.5,1,2,5] # crest widths
            cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='turbo'),cax=cax,ticks=ticks,format='%.1f')
            cbar.ax.set_ylabel(f'Crest Width (m/m)', rotation=270, labelpad=15)
        if graphs[i]=='D':
            ticks=[1,1.5,2,2.5,3]
            cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='turbo'),cax=cax,ticks=ticks,format='%.1f')
            cbar.ax.set_ylabel(f'Slope angle (1:x)', rotation=270, labelpad=15)


ax.invert_yaxis()
A.set(ylabel="Effective depth (m/m)")#, title=f'cot(θ) = {cot}, crest width = {cw}'",
C.set(xlabel="Topographic effect - E (Ω/Ω)",ylabel="Effective depth (m/m)",xscale='log')
D.set(xlabel="Topographic effect - E (Ω/Ω)",xscale='log')
#ax.tick_params(which='both', length=2)

A.xaxis.set_minor_locator(ticker.FixedLocator(locs=[0.8,0.9,1.1,1.2,1.3,1.4,1.5]))
A.xaxis.set_minor_formatter(ticker.ScalarFormatter(useMathText=True))
A.xaxis.set_major_locator(ticker.FixedLocator(locs=[1]))
A.xaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))

B.xaxis.set_minor_locator(ticker.FixedLocator(locs=[0.7,0.8,0.9,2,3,4,5,6,7,8,9]))
B.xaxis.set_minor_formatter(ticker.FixedFormatter(['0.7','','','2','','','5','','','8','']))
B.xaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
B.xaxis.set_major_locator(ticker.FixedLocator(locs=[1,10]))

A.tick_params(which='both', length=2)
B.tick_params(which='both', length=2)
C.tick_params(which='both', length=2)
D.tick_params(which='both', length=2)

A.text(0.98,0.02,'RC = 100\nSlope angle 1:2', ha='right', va='bottom',transform=A.transAxes).set_bbox(dict(facecolor='white', alpha=0.5,linewidth=0,pad=0.1))
B.text(0.98,0.98,'RC = 0.01\nSlope angle 1:2', ha='right', va='top',transform=B.transAxes).set_bbox(dict(facecolor='white', alpha=0.5,linewidth=0,pad=0.1))
C.text(0.98,0.02,'RC = 100\nCrest Width 0.42', ha='right', va='bottom',transform=C.transAxes).set_bbox(dict(facecolor='white', alpha=0.5,linewidth=0,pad=0.1))
D.text(0.98,0.98,'RC = 0.01\nCrest Width 1.05', ha='right', va='top',transform=D.transAxes).set_bbox(dict(facecolor='white', alpha=0.5,linewidth=0,pad=0.1))

A.text(0.1, 0.5, 'E2',ha='right', size=10,transform=A.transAxes)
B.text(0.1, 0.7, 'E3',ha='right', size=10,transform=B.transAxes)
C.text(0.2, 0.47, 'E2',ha='right', size=10,transform=C.transAxes)
D.text(0.12, 0.45, 'E3',ha='right', size=10,transform=D.transAxes)


#Save figure
plt.savefig('Figures/Figrue 7 Supplementary curves.svg',format='svg',transparent=True,bbox_inches='tight',pad_inches=0.1)

## Figure 7 variations in resistivity

In [ ]:
rc_log= np.array(resistivity_contrasts_log)

In [ ]:
fig, ((A, B), (C, D)) = plt.subplots(2, 2,sharex=True, sharey=True)
plt.rcParams.update({'font.size': 10})
fig.set_size_inches(8, 5)
plt.subplots_adjust(wspace=0.08, hspace=0.08)

# crest width options
#[0.17 0.21 0.26 0.33 0.42 0.53 0.66 0.83 1.05 1.32 1.67 2.1  2.64 3.32 4.19 5.27] 

cw=[2.1,2.1,0.17,0.17] #filter by parameter
cot=[3.0,3.0,3.0,3.0]
array_list=['w','dd','w','dd'] # array type
graphs=['A','B','C','D'] # axis
arrays={'w':'Wenner','dd':'Dipole-Dipole'} #dictionary lookup for labels

for i,ax in enumerate([A,B,C,D]):
    graph=graphs[i]    
    geo_filter=(data_n1['cw']==cw[i]) & (data_n1['cot']==cot[i]) & (data_n1['array'] == array_list[i])
    filtered = data_n1[geo_filter]
    
    #Plot line data
    cMin=min(resistivity_contrasts)
    cMax=max(resistivity_contrasts)
    print(cMin,cMax)

    rc_log= np.array(resistivity_contrasts_log)
    colour = iter(plt.cm.turbo((rc_log-min(rc_log))/(max(rc_log)-min(rc_log))))

    var=resistivity_contrasts
    for v,c in zip(var, colour):
        #filtered_cw=filtered_cw.groupby('doi').mean().reset_index()
        x=filtered[f'E_{v}']
        y=filtered['doi']
        spl = CubicSpline(y, x)
        y_new=np.logspace(log10(y.min()),log10(y.max()),num=1001)
        ax.plot(spl(y_new),y_new,label=i,color=c,zorder=1)
        ax.scatter(x=x,y=y,s=0.1,c='0',alpha=0.3,zorder=2)
    
    #Plot scatter data
    #ax_plot=ax.scatter(x=filtered[f'E_{res}'],y=filtered['doi'],s=0.1,c=filtered[variable[i]],cmap='viridis')
    ax_plot=ax.scatter(x=filtered[f'E_{v}'],y=filtered['doi'],s=0.1,c='0',alpha=0.3,zorder=2)

    ax.set_yscale('log')
    ax.invert_yaxis()
    ax.grid(which='major')
    ax.grid(which='minor',lw=0.2)
    majors = ["0.1","1","10"]
    ax.yaxis.set_major_locator(ticker.FixedLocator(locs=[0.1,1,10]))
    ax.yaxis.set_major_formatter(ticker.FixedFormatter(majors))
    ax.text(-0.07, 1, graphs[i], size=10,transform=ax.transAxes, weight='bold')
    ax.text(0.98,0.98,f'{arrays[array_list[i]]}\nCrest Width={cw[i]}\nAngle=1:{cot[i]:g}', ha='right', va='top',size=9,transform=ax.transAxes).set_bbox(dict(facecolor='white', alpha=0.5,linewidth=0,pad=0.1))
    
#plot colour bar
ax_all=[A,B,C,D]
ticks=[0.01,0.02,0.05,0.1,0.2,0.5,1,2,5,10,20,50,100] # resistivitiy contrasts
norm=mpl.colors.LogNorm(vmin=cMin, vmax=cMax)
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='turbo'),ax=ax_all,ticks=ticks,format='%.3g',pad=0.01, aspect=40)
cbar.ax.set_ylabel('Resistivity Contrast (Ωm/Ωm)', rotation=270, labelpad=10)

#plot lables
A.text(0.8, 0.3, 'E1',ha='left', size=10,transform=A.transAxes)
B.text(0.8, 0.3, 'E1',ha='left', size=10,transform=B.transAxes)
B.text(0.1, 0.7, 'E3',ha='right', size=10,transform=B.transAxes)
C.text(0.9, 0.3, 'E1',ha='left', size=10,transform=C.transAxes)
C.text(0.1, 0.47, 'E2',ha='right', size=10,transform=C.transAxes)
D.text(0.9, 0.3, 'E1',ha='left', size=10,transform=D.transAxes)
D.text(0.1, 0.47, 'E2',ha='right', size=10,transform=D.transAxes)
D.text(0.12, 0.63, 'E3',ha='right', size=10,transform=D.transAxes)

#plot axis labels
ax.invert_yaxis()
A.set(ylabel="Effective depth (m/m)")#, title=f'cot(θ) = {cot}, crest width = {cw}'",
C.set(xlabel="Topographic effect - E (Ω/Ω)",ylabel="Effective depth (m/m)",xscale='log')
D.set(xlabel="Topographic effect - E (Ω/Ω)",xscale='log')

A.tick_params(which='both', length=2)
B.tick_params(which='both', length=2)
C.tick_params(which='both', length=2)
D.tick_params(which='both', length=2)

A.xaxis.set_minor_locator(ticker.FixedLocator(locs=[0.7,0.8,0.9,2,3,4,5,6,7,8,9]))
A.xaxis.set_minor_formatter(ticker.FixedFormatter(['0.7','','','2','','','5','','','','9']))
A.xaxis.set_major_formatter(FormatStrFormatter('%.1g'))
#A.xaxis.set_minor_formatter(FormatStrFormatter('%.1g'))
#B.xaxis.set_major_formatter(FormatStrFormatter('%.1g'))
#B.xaxis.set_minor_formatter(FormatStrFormatter('%.1g'))

#Save figure
plt.savefig('Figures/Figrue 5 topo and res effect line interp.svg',format='svg',transparent=True,pad_inches=0.1,bbox_inches='tight')

# Extract max and min values from souding curves

In [ ]:
columns=['cw','cot','bw','area','dd_err','dd_doi','w_err','w_doi']
max_errors=pd.DataFrame(columns=columns)
min_errors=pd.DataFrame(columns=columns)
res=1

crest_widths=sorted(data_n1['cw'].unique())
print(f'cw:\n{crest_widths}')
angles=sorted(data_n1['cot'].unique())
print(f'cot:\n{angles}')

for cw in crest_widths:
    for cot in angles:
        #embankment properties
        height=1
        bw=cw+height*cot*2
        radius=(height**2+(cw/2+cot*height)**2)**0.5 # distance from electrode line to toe of embankment
        theta=math.acos(height/radius)
        area=height*(cw+cot*height)
        #print('height: ',height,'cw: ', cw,'cot: ',cot,'radius: ',radius,'theta: ', theta)
        #area_circle=theta*radius**2+(radius*height)/(math.sin(math.pi/2-theta))
        #print('area_circle: ',area_circle,'area: ', area)
        
        #filter data based on geometry and array type
        dd_filter=(data_n1['cot']==cot) & (data_n1['cw']==cw) & (data_n1['array']=='dd')
        dd_data=data_n1[dd_filter]
        w_filter=(data_n1['cot']==cot) & (data_n1['cw']==cw) & (data_n1['array']=='w')
        w_data=data_n1[w_filter]
        
        #fit cubic spline
        #dipole diple
        x_dd=dd_data[f'E_{res}']
        y_dd=dd_data['doi']
        spl_dd = CubicSpline(y_dd, x_dd)
        y_dd_new = np.logspace(log10(y_dd.min()),log10(y_dd.max()),num=1001)
        x_dd_new = spl_dd(y_dd_new)
        dd_dict=dict(zip(x_dd_new, y_dd_new))
        dd_min_effect=min(x_dd_new)
        dd_depth_min=dd_dict[dd_min_effect]
        dd_max_effect=max(x_dd_new)
        dd_depth=dd_dict[dd_max_effect]
        #wenner
        x_w=w_data[f'E_{res}']
        y_w=w_data['doi']
        spl_w = CubicSpline(y_w, x_w)
        y_w_new = np.logspace(log10(y_w.min()),log10(y_w.max()),num=1001)
        x_w_new = spl_w(y_w_new)
        w_dict=dict(zip(x_w_new, y_w_new))
        w_min_effect=min(x_w_new)
        w_depth_min=w_dict[w_min_effect]
        w_max_effect=max(x_w_new)
        w_depth=w_dict[w_max_effect]
        
        #cobine datasets
        cw=float(cw)
        cot=round(math.degrees(np.arctan(1/cot)),2)# cot(θ) to degrees 0 decimal place.
        semi_circle=math.pi*w_depth
        temp_min=pd.DataFrame([[cw,cot,bw,area,dd_min_effect,dd_depth_min,w_min_effect,w_depth_min]],columns=columns)
        temp_max=pd.DataFrame([[cw,cot,bw,area,dd_max_effect,dd_depth,w_max_effect,w_depth]],columns=columns)
        
        min_errors=pd.concat([min_errors,temp_min])
        max_errors=pd.concat([max_errors,temp_max])
        

corr=max_errors.corr(method='spearman')
corr_log=np.log10(max_errors).corr()
corr_min=min_errors.corr()
corr_min_log=np.log10(min_errors).corr()
print(max_errors)


### Plot maximum values Figure 8

In [ ]:
fig, axs = plt.subplots(2, 4)
array='dd'
sign='max' # 'max or min'
r_cw=round(corr.at[f'{array}_err','cw'],2)
r_bw = round(corr.at[f'{array}_err','bw'],2)
r_cot = round(corr.at[f'{array}_err','cot'],2)
r_area = round(corr.at[f'{array}_err','area'],2)

axs[0,0].scatter(max_errors['cw'], max_errors[f'{array}_err'],s=1.5,c='k')
#axs[0,0].set_xlabel('Crest Width [m]')
axs[0,0].text(0.98, 0.98,f'$r_s$={r_cw}', ha='right', va='top',transform=axs[0,0].transAxes)
axs[0,0].set_ylabel('Max topographic effect (Ω/Ω)')
axs[0,1].scatter(max_errors['cot'], max_errors[f'{array}_err'],s=1.5,c='k')
#axs[0,1].set_xlabel('Angle [cot(Θ)]')
axs[0,1].text(0.02, 0.98,f'$r_s$={r_cot}', ha='left', va='top',transform=axs[0,1].transAxes)
axs[0,2].scatter(max_errors['bw'], max_errors[f'{array}_err'],s=1.5,c='k')
#axs[0,2].set_xlabel('Base Width [m]')
axs[0,2].text(0.98, 0.98,f'$r_s$={r_bw}', ha='right', va='top',transform=axs[0,2].transAxes)
axs[0,3].scatter(max_errors['area'], max_errors[f'{array}_err'],s=1.5,c='k')
#axs[0,3].set_xlabel('Area X-Section [m]')
axs[0,3].text(0.98, 0.98,f'$r_s$={r_area}', ha='right', va='top',transform=axs[0,3].transAxes)

r_cw=round(corr.at[f'{array}_doi','cw'],2)
r_bw = round(corr.at[f'{array}_doi','bw'],2)
r_cot = round(corr.at[f'{array}_doi','cot'],2)
r_area = round(corr.at[f'{array}_doi','area'],2)

axs[1,0].scatter(max_errors['cw'], max_errors[f'{array}_doi'],s=1.5,c='k')
axs[1,0].set_xlabel('Crest width (m/m)')
axs[1,0].text(0.02,0.98,f'$r_s$={r_cw}', ha='left', va='top',transform=axs[1,0].transAxes)
axs[1,0].set_ylabel('Depth of max \ntopographic effect (m/m)')
axs[1,1].scatter(max_errors['cot'], max_errors[f'{array}_doi'],s=1.5,c='k')
axs[1,1].set_xlabel('Slope angle (1:x)')
axs[1,1].text(0.98,0.98,f'$r_s$={r_cot}', ha='right', va='top',transform=axs[1,1].transAxes)
axs[1,2].scatter(max_errors['bw'], max_errors[f'{array}_doi'],s=1.5,c='k')
axs[1,2].set_xlabel('Base Width (m/m)')
axs[1,2].text(0.02,0.98,f'$r_s$={r_bw}', ha='left', va='top',transform=axs[1,2].transAxes)
axs[1,3].scatter(max_errors['area'], max_errors[f'{array}_doi'],s=1.5,c='k')
axs[1,3].set_xlabel('Area X-Section (m\u00b2/m\u00b2)')
axs[1,3].text(0.02,0.98,f'$r_s$={r_area}', ha='left', va='top',transform=axs[1,3].transAxes)

#for ax in axs.flat:
   # ax.set(xlabel='x-label', ylabel='y-label')
for ax in axs.flat:
    ax.label_outer()
fig.set_size_inches(8, 5)
plt.subplots_adjust(wspace=0.1, hspace=0.08)
plt.savefig(f'Figures/Correlation plot max {res} {array}.svg',format='svg',transparent=True,bbox_inches='tight')



### Plot minimum values

In [ ]:
fig, axs = plt.subplots(2, 4)
array='dd'
r_cw=round(corr_min.at[f'{array}_err','cw'],2)
r_bw = round(corr_min.at[f'{array}_err','bw'],2)
r_cot = round(corr_min.at[f'{array}_err','cot'],2)
r_area = round(corr_min.at[f'{array}_err','area'],2)

axs[0,0].scatter(min_errors['cw'], min_errors[f'{array}_err'],s=1.5,c='k')
#axs[0,0].set_xlabel('Crest Width [m]')
axs[0,0].text(0.98, 0.98,f'r={r_cw}', ha='right', va='top',transform=axs[0,0].transAxes)
axs[0,0].set_ylabel('Min topographic effect [%]')
axs[0,1].scatter(min_errors['cot'], min_errors[f'{array}_err'],s=1.5,c='k')
#axs[0,1].set_xlabel('Angle [cot(Θ)]')
axs[0,1].text(0.02, 0.98,f'r={r_cot}', ha='left', va='top',transform=axs[0,1].transAxes)
axs[0,2].scatter(min_errors['bw'], min_errors[f'{array}_err'],s=1.5,c='k')
#axs[0,2].set_xlabel('Base Width [m]')
axs[0,2].text(0.98, 0.98,f'r={r_bw}', ha='right', va='top',transform=axs[0,2].transAxes)
axs[0,3].scatter(min_errors['area'], min_errors[f'{array}_err'],s=1.5,c='k')
#axs[0,3].set_xlabel('Area X-Section [m]')
axs[0,3].text(0.98, 0.98,f'r={r_area}', ha='right', va='top',transform=axs[0,3].transAxes)

r_cw=round(corr_min.at[f'{array}_doi','cw'],2)
r_bw = round(corr_min.at[f'{array}_doi','bw'],2)
r_cot = round(corr_min.at[f'{array}_doi','cot'],2)
r_area = round(corr_min.at[f'{array}_doi','area'],2)

axs[1,0].scatter(min_errors['cw'], min_errors[f'{array}_doi'],s=1.5,c='k')
axs[1,0].set_xlabel('Crest width [m]')
axs[1,0].text(0.02,0.98,f'r={r_cw}', ha='left', va='top',transform=axs[1,0].transAxes)
axs[1,0].set_ylabel('Depth of min \ntopographic effect [m]')
axs[1,1].scatter(min_errors['cot'], min_errors[f'{array}_doi'],s=1.5,c='k')
axs[1,1].set_xlabel('Slope angle [Θ]')
axs[1,1].text(0.98,0.98,f'r={r_cot}', ha='right', va='top',transform=axs[1,1].transAxes)
axs[1,2].scatter(min_errors['bw'], min_errors[f'{array}_doi'],s=1.5,c='k')
axs[1,2].set_xlabel('Base Width [m]')
axs[1,2].text(0.02,0.98,f'r={r_bw}', ha='left', va='top',transform=axs[1,2].transAxes)
axs[1,3].scatter(min_errors['area'], min_errors[f'{array}_doi'],s=1.5,c='k')
axs[1,3].set_xlabel('Area X-Section [m\u00b2]')
axs[1,3].text(0.02,0.98,f'r={r_area}', ha='left', va='top',transform=axs[1,3].transAxes)

#for ax in axs.flat:
   # ax.set(xlabel='x-label', ylabel='y-label')
for ax in axs.flat:
    ax.label_outer()
fig.set_size_inches(8, 5)
plt.subplots_adjust(wspace=0.1, hspace=0.08)
plt.savefig(f'Figures/Correlation plot min {res} {array}.svg',format='svg')

In [ ]:
def fit_model(geometry,error):
    #function take a geometry coloumn and the error/depth coloumn, fits a line in loglog space and then exports the fit
    geometry_log=np.log10(geometry)
    error_log=np.log10(error)
    fit = np.polyfit(geometry_log, error_log, 1)
    geometry_sorted=np.array(sorted(geometry.unique()))# list of unique areas
    err_fit=(10**fit[1])*(geometry_sorted)**fit[0]
    return [geometry_sorted,err_fit]


In [ ]:

fig, axs = plt.subplots(2, 4,sharex='col',sharey='row')
fig.set_size_inches(8, 5)
plt.subplots_adjust(wspace=0.1, hspace=0.08)

array='dd'
c='k'
opacity=1

#plot data
for i,item in enumerate(['cw','cot','bw','area']):
    
    #extract data for geometry
    geometry=max_errors[f'{item}']
    error=np.log10(max_errors[f'{array}_err'])
    error_depth=max_errors[f'{array}_doi']
    
    #fit data
    fit_error=fit_model(geometry,error=error)
    fit_depth=fit_model(geometry,error=error_depth)
    
    #plot data
    axs[0,i].scatter(geometry, error,s=1.5,c=c,alpha=opacity)
    axs[1,i].scatter(geometry, error_depth,s=1.5,c=c,alpha=opacity)
    
    #plot line of best fit
    #axs[0,i].plot(fit_error[0],fit_error[1])
    #axs[1,i].plot(fit_depth[0],fit_depth[1])
    
    #plot correlation coef.
    cor_err=round(np.corrcoef(np.log10(geometry), np.log10(error))[0,1],2)
    cor_doi=round(np.corrcoef(np.log10(geometry), np.log10(error_depth))[0,1],2)
    
    if i==1:
        axs[0,i].text(0.02, 0.98,f'r={cor_err}', ha='left', va='top',transform=axs[0,i].transAxes)
        axs[1,i].text(0.98, 0.98,f'r={cor_doi}', ha='right', va='top',transform=axs[1,i].transAxes)
    else:    
        axs[0,i].text(0.98, 0.98,f'r={cor_err}', ha='right', va='top',transform=axs[0,i].transAxes)
        axs[1,i].text(0.02, 0.98,f'r={cor_doi}', ha='left', va='top',transform=axs[1,i].transAxes)

#axis formating

for ax in axs.flat:
    ax.loglog()
    ax.label_outer()
    ax.tick_params(which='both', length=2)

#set labels
axs[0,0].set_ylabel('Max topographic\neffect [Ωm/Ωm]')
axs[1,0].set_xlabel('Crest width [m/m]')
axs[1,0].set_ylabel('Depth of\nmax effect [m/m]')
axs[1,1].set_xlabel('Slope angle [Θ]')
axs[1,2].set_xlabel('Base Width [m/m]')
axs[1,3].set_xlabel('Area X-Section [m\u00b2/m\u00b2]')

#top row y
axs[0,0].set_yticks([1])
axs[0,0].set_yticklabels(['1'])
axs[0,0].yaxis.set_minor_locator(ticker.FixedLocator(locs=np.log10([1.1,1.3,1.5,1.7,1.9])))
axs[0,0].yaxis.set_minor_formatter(ticker.FixedFormatter(['1.1','1.3','1.5','1.7','1.9']))

#bottom row y
axs[1,0].set_yticks([1])
axs[1,0].set_yticklabels(['1'])
axs[1,0].yaxis.set_minor_locator(ticker.FixedLocator(locs=[0.6,0.7,0.8,0.9,2,3,4]))
axs[1,0].yaxis.set_minor_formatter(ticker.FixedFormatter(['0.6','','','','2','3','4']))    

#x axis
axs[1,0].set_xticks([1])
axs[1,0].set_xticklabels(['1'])
axs[1,0].xaxis.set_minor_locator(ticker.FixedLocator(locs=[0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,2,3,4,5,6,7,8,9]))
axs[1,0].xaxis.set_minor_formatter(ticker.FixedFormatter(['0.2','','','0.5','','','','','2','','','5','','','','']))
    
axs[1,1].xaxis.set_minor_locator(ticker.FixedLocator(locs=[20,30,40]))
axs[1,1].xaxis.set_minor_formatter(ticker.FixedFormatter(['20','30','40']))

axs[1,2].set_xticks([10])
axs[1,2].set_xticklabels(['10'])
axs[1,2].xaxis.set_minor_locator(ticker.FixedLocator(locs=[2,3,4,5,6,7,8,9]))
axs[1,2].xaxis.set_minor_formatter(ticker.FixedFormatter(['2','3','','5','','','','']))

axs[1,3].set_xticks([1])
axs[1,3].set_xticklabels(['1'])
axs[1,3].xaxis.set_minor_locator(ticker.FixedLocator(locs=[2,3,4,5,6,7,8,9]))
axs[1,3].xaxis.set_minor_formatter(ticker.FixedFormatter(['2','3','','5','','','','9']))

#Export figure
plt.savefig(f'Figures/Correlation plot log max {res} {array}.svg',format='svg',transparent=True,bbox_inches='tight')


## Figure 13 topogrpahic and subsurface resistivity effect

In [ ]:
columns=['geo_id','cw','cot','bw','area']
max_errors=pd.DataFrame(columns=columns)
res=1

crest_widths=sorted(data_n1['cw'].unique())
print(f'cw:\n{crest_widths}')
angles=sorted(data_n1['cot'].unique())
print(f'cot:\n{angles}')


for cw in crest_widths:
    for cot in angles:
        #embankment properties
        height=1
        bw=cw+height*cot*2
        radius=(height**2+(cw/2+cot*height)**2)**0.5 # distance from electrode line to toe of embankment
        theta=math.acos(height/radius)
        area=height*(cw+cot*height)
        slope_angle=round(math.degrees(np.arctan(1/cot)),2)# cot(θ) to degrees 0 decimal place.
        geo_id=f'{cw}_{cot}'
        properties=pd.DataFrame(data=[[geo_id,cw,slope_angle,bw,area]],columns=columns)

        #print('height: ',height,'cw: ', cw,'cot: ',cot,'radius: ',radius,'theta: ', theta)
        #area_circle=theta*radius**2+(radius*height)/(math.sin(math.pi/2-theta))
        #print('area_circle: ',area_circle,'area: ', area)

        #filter data based on geometry and array type
        dd_filter=(data_n1['cot']==cot) & (data_n1['cw']==cw) & (data_n1['array']=='dd')
        dd_data=data_n1[dd_filter]
        w_filter=(data_n1['cot']==cot) & (data_n1['cw']==cw) & (data_n1['array']=='w')
        w_data=data_n1[w_filter]

        #fit cubic spline
        #dipole diple
        for res in resistivity_contrasts:
            x_dd=dd_data[f'E_{res}']
            y_dd=dd_data['doi']
            spl_dd = CubicSpline(y_dd, x_dd)
            y_dd_new = np.logspace(log10(y_dd.min()),log10(y_dd.max()),num=1001)
            x_dd_new = spl_dd(y_dd_new)
            dd_dict=dict(zip(x_dd_new, y_dd_new))
            dd_max_effect=max(x_dd_new)
            dd_depth=dd_dict[dd_max_effect]
            #wenner
            x_w=w_data[f'E_{res}']
            y_w=w_data['doi']
            spl_w = CubicSpline(y_w, x_w)
            y_w_new = np.logspace(log10(y_w.min()),log10(y_w.max()),num=1001)
            x_w_new = spl_w(y_w_new)
            w_dict=dict(zip(x_w_new, y_w_new))
            w_max_effect=max(x_w_new)
            w_depth=w_dict[w_max_effect]

            #cobine datasets
            properties[[f'dd_max_effect_E_{res}',f'dd_depth_e_{res}',f'w_max_effect_E_{res}',f'w_depth_e_{res}']]=[dd_max_effect,dd_depth,w_max_effect,w_depth]
            #semi_circle=math.pi*w_depth
        #temp=pd.DataFrame([[cw,cot,bw,area,dd_max_effect,dd_depth,w_max_effect,w_depth]],columns=columns)
        max_errors=pd.concat([max_errors,properties])
corr=max_errors.corr()
#corr_log=np.log10(max_errors).corr()
print(max_errors[max_errors.geo_id=='2.1_2.0'][['w_depth_e_0.1','w_depth_e_1','w_depth_e_10']])


In [ ]:
def fit_model_3rd(geometry,error):
    #function take a geometry coloumn and the error/depth coloumn, fits a line in loglog space and then exports the fit
    geometry_log=np.log10(geometry)
    error_log=np.log10(error)
    fit = np.polyfit(geometry_log, error_log, 3)
    
    geometry_sorted_log=np.array(sorted(geometry_log.unique()))# list of unique areas
    err_fit=np.poly1d(fit)(geometry_sorted_log)
    
    #err_fit=(fit[0])*(geometry_sorted)**3+fit[1]*(geometry_sorted)**2+fit[2]*(geometry_sorted)+fit[3]
    return [10**geometry_sorted_log,10**err_fit]

def fit_model_2nd(geometry,error):
    #function take a geometry coloumn and the error/depth coloumn, fits a line in loglog space and then exports the fit
    fit = np.polyfit(geometry, error, 1)
    geometry_sorted=np.array(sorted(geometry.unique()))# list of unique areas
    err_fit=np.poly1d(fit)(geometry_sorted)
    
    #err_fit=(fit[0])*(geometry_sorted)**3+fit[1]*(geometry_sorted)**2+fit[2]*(geometry_sorted)+fit[3]
    return [geometry_sorted,err_fit]

def fit_sqrt_x(geometry,error):
    def f(x, a,c):
        return a*x**0.5 + c
    params, covariance = curve_fit(f, geometry, error)
    geometry_sorted=np.array(sorted(geometry.unique()))# list of unique areas

    print(f'params: {params}')
    err_fit = params[0]*geometry_sorted**0.5+params[1]
    return [geometry_sorted,err_fit]
    
    

In [ ]:
#Plot line data
fig, [A,B,C] = plt.subplots(1,3,constrained_layout = True)#sharex='col', sharey=True)
fig.set_size_inches(8, 4)
C.sharey(B)


rc_log= np.array(resistivity_contrasts_log)
colour = iter(plt.cm.turbo((rc_log-min(rc_log))/(max(rc_log)-min(rc_log))))

A_l=[]
B_l=[]
C_l=[]

graphs=['A','B','C']
for res,c in zip(resistivity_contrasts, colour):
    A.scatter(max_errors['area'],max_errors[f'dd_max_effect_E_{res}'],label=res,color=c,s=1,alpha=0.5)
    #A.scatter(max_errors['area'],max_errors[f'w_max_effect_E_{res}'],label=res,color=c,s=1)
    fit_A=fit_model_3rd(geometry=max_errors['area'],error=max_errors[f'dd_max_effect_E_{res}'])
    A.plot(fit_A[0],fit_A[1],color=c,label=res)
    
    if res <= 100:
        B.scatter(max_errors['area'],max_errors[f'dd_depth_e_{res}'],label=res,color=c,s=1,alpha=0.5)
        #B.scatter(max_errors['area'],max_errors[f'w_depth_e_{res}'],label=res,color=c,s=1)
        fit_B=fit_sqrt_x(geometry=max_errors['area'],error=max_errors[f'dd_depth_e_{res}'])
        B.plot(fit_B[0],fit_B[1],color=c,label=res)
    if res >= 0.01:
        C.scatter(max_errors['cw'],max_errors[f'dd_depth_e_{res}'],label=res,color=c,s=1,alpha=0.5)
        #C.scatter(max_errors['cw'],max_errors[f'w_depth_e_{res}'],label=res,color=c,s=1)
        fit_C=fit_model_3rd(geometry=max_errors['cw'],error=max_errors[f'dd_depth_e_{res}'])
        C.plot(fit_C[0],fit_C[1],color=c,label=res)
    

A.set(xlabel='Embankment area [m\u00b2/m\u00b2]', ylabel='Topographic effect [Ωm/Ωm]')#,xscale='log',yscale='log')
B.set(xlabel='Embankment area [m\u00b2/m\u00b2]', ylabel='Depth of max error [m/m]')#,xscale='log',yscale='log')
C.set(xlabel='Crest width [m/m]')#,xscale='log',yscale='log')

#Plot grid lines
for i,ax in enumerate([A,B,C]):
    #ax.grid(which='major')
    #ax.grid(which='minor',lw=0.2)
    ax.text(-0.07, 1, graphs[i], size=10,transform=ax.transAxes, weight='bold')




ax_all=[A,B,C]
cMin=min(resistivity_contrasts)
cMax=max(resistivity_contrasts)
norm=mpl.colors.LogNorm(vmin=cMin, vmax=cMax)
ticks=[0.01,0.1,1,10,100] # resistivities
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='turbo'),ax=ax_all,ticks=ticks,format='%.3g',pad=0.01, aspect=40)
cbar.ax.set_ylabel(f'Resistivity contrast', rotation=270, labelpad=10)

plt.savefig('Figures/Max topo effect.svg',format='svg',transparent=True,pad_inches=0.1,bbox_inches = 'tight')

In [ ]:
fig, A = plt.subplots(1,1,constrained_layout = True)#sharex='col', sharey=True)
fig.set_size_inches(3.5, 3.5)


rc_log= np.array(resistivity_contrasts_log)
colour = iter(plt.cm.turbo((rc_log-min(rc_log))/(max(rc_log)-min(rc_log))))

A_l=[]


graphs=['A','B','C']
for res,c in zip(resistivity_contrasts, colour):
    A.scatter(max_errors['area'],max_errors[f'dd_max_effect_E_{res}'],label=res,color=c,s=1,alpha=0.5)
    #A.scatter(max_errors['area'],max_errors[f'w_max_effect_E_{res}'],label=res,color=c,s=1)
    fit_A=fit_model_3rd(geometry=max_errors['area'],error=max_errors[f'dd_max_effect_E_{res}'])
    A.plot(fit_A[0],fit_A[1],color=c,label=res)
    

A.set(xlabel='Embankment cross-sectional area (m\u00b2/m\u00b2)', ylabel='Topographic effect - E (Ω/Ω)',yscale='log')
A.grid(which='both')
cMin=min(resistivity_contrasts)
cMax=max(resistivity_contrasts)
norm=mpl.colors.LogNorm(vmin=cMin, vmax=cMax)
ticks=[0.01,0.1,1,10,100] # resistivities
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='turbo'),ax=A,ticks=ticks,format='%.3g',pad=0.05, aspect=30)
cbar.ax.set_ylabel(f'Resistivity contrast - RC (Ωm/Ωm)', rotation=270, labelpad=10)

A.tick_params(which='both', length=2)
A.yaxis.set_major_locator(ticker.FixedLocator(locs=[1,10]))
A.yaxis.set_major_formatter(ticker.FixedFormatter(['1','10']))
A.yaxis.set_minor_locator(ticker.FixedLocator(locs=[0.9,1.1,2,3,4,5,6,7,8,9,15]))
A.yaxis.set_minor_formatter(ticker.FixedFormatter(['','1.1','2','','','5','','','','','15']))
#A.yaxis.set_major_formatter(FormatStrFormatter('%.1g'))


plt.savefig('Figures/Max topo effect with RC.svg',format='svg',transparent=True,pad_inches=0.1,bbox_inches = 'tight')

In [ ]:
fig, A = plt.subplots(1,1,constrained_layout = True)#sharex='col', sharey=True)
fig.set_size_inches(3.5, 3.5)


rc_log= np.array(resistivity_contrasts_log)
colour = iter(plt.cm.turbo((rc_log-min(rc_log))/(max(rc_log)-min(rc_log))))

A_l=[]


graphs=['A','B','C']
for res,c in zip(resistivity_contrasts, colour):
    A.scatter(max_errors['area'],max_errors[f'dd_max_effect_E_{res}'],label=res,color=c,s=1,alpha=0.5)
    #A.scatter(max_errors['area'],max_errors[f'w_max_effect_E_{res}'],label=res,color=c,s=1)
    fit_A=fit_model_3rd(geometry=max_errors['area'],error=max_errors[f'dd_max_effect_E_{res}'])
    A.plot(fit_A[0],fit_A[1],color=c,label=res)
    

A.set(xlabel='Embankment cross-sectional area (m\u00b2/m\u00b2)', ylabel='Topographic effect - E (Ω/Ω)',ylim=[0, 15])
A.grid(which='both')
cMin=min(resistivity_contrasts)
cMax=max(resistivity_contrasts)
norm=mpl.colors.LogNorm(vmin=cMin, vmax=cMax)
ticks=[0.01,0.1,1,10,100] # resistivities
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='turbo'),ax=A,ticks=ticks,format='%.3g',pad=0.05, aspect=30)
cbar.ax.set_ylabel(f'Resistivity contrast - RC (Ωm/Ωm)', rotation=270, labelpad=10)

In [ ]:
#label formating
#plot A
majors = ["1","10","50","90"]
A.yaxis.set_major_locator(ticker.FixedLocator(locs=[1,10,50,90]))
A.yaxis.set_major_formatter(ticker.FixedFormatter(majors))
#plot A,B
for ax in [A,B]:
    minorx=['2','5','9']
    ax.xaxis.set_minor_locator(ticker.FixedLocator(locs=[2,5,9]))
    ax.xaxis.set_minor_formatter(ticker.FixedFormatter(minorx))
#plot B
for ax in [B]:
    majory = ["1","10"]
    ax.yaxis.set_major_locator(ticker.FixedLocator(locs=[1,10]))
    ax.yaxis.set_major_formatter(ticker.FixedFormatter(majory))
#plot C
C.set_xticks([1])
C.set_xticklabels(['1'])
C.xaxis.set_minor_locator(ticker.FixedLocator(locs=[0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,2,3,4,5]))
C.xaxis.set_minor_formatter(ticker.FixedFormatter(['0.2','','','0.5','','','','','2','','','5']))
C.yaxis.set_tick_params(labelleft=False)

## Attempt 2 different data structure

In [ ]:
columns=['geo_id','cw','cot','bw','area']
max_errors=pd.DataFrame(columns=columns)

crest_widths=sorted(data_n1['cw'].unique())
angles=sorted(data_n1['cot'].unique())


for cw in crest_widths:
    for cot in angles:
        #embankment properties
        height=1
        bw=cw+height*cot*2
        area=height*(cw+cot*height)
        slope_angle=round(math.degrees(np.arctan(1/cot)),2)# cot(θ) to degrees 0 decimal place.
        geo_id=f'{cw}_{cot}'
        properties=pd.DataFrame(data=[[geo_id,cw,slope_angle,bw,area]],columns=columns)


        #filter data based on geometry and array type
        data_filter=(data_n1['cot']==cot) & (data_n1['cw']==cw)
        filtered_data=data_n1[data_filter].reset_index(drop=True)

        for res in resistivity_contrasts:
            for array in ['dd','w']:
                array_data=filtered_data[(filtered_data['array']==array)]#filter by array type
                x=array_data[f'E_{res}']
                y=array_data['doi']
                spl = CubicSpline(y, x) #fit spline through each curve
                ynew = np.logspace(log10(y.min()),log10(y.max()),num=1001)
                xnew = spl(y_dd_new)

                new_dict=dict(zip(xnew, ynew))
                
                max_effect=max(xnew)
                depth=new_dict[max_effect]

                #cobine datasets
                properties[['res','array','max_effect','depth']]=[res,array,max_effect,depth]
                max_errors=pd.concat([max_errors,properties])
            #semi_circle=math.pi*w_depth
        #temp=pd.DataFrame([[cw,cot,bw,area,dd_max_effect,dd_depth,w_max_effect,w_depth]],columns=columns)
        
max_errors.reset_index(drop=True, inplace=True)
max_errors

In [ ]:
group=max_errors.groupby(['res','area']).mean().reset_index()
group['res_log']=np.log10(group['res'])
group['max_effect_log']=np.log10(group['max_effect'])

x=(group['res'].unique())
y=np.asarray(group['area'].unique())
z=np.log10(group['max_effect'])
matrix=group.pivot(index='res',columns='area',values='max_effect_log')
fig, ax=plt.subplots(1,1,figsize=(4, 4))
ax.pcolormesh(y,x,matrix,cmap='turbo')
ax.contour(y,x,matrix,cmap='turbo')

ax.set(ylabel='Resistivity Contrast',xlabel='Area [m/m]',yscale='log')

norm=mpl.colors.LogNorm(vmin=min(group['max_effect']), vmax=max(group['max_effect']))
ticks=[0.3,1,5,10,50,90] # resistivities
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='turbo'),ax=ax,ticks=ticks,format='%.3g',pad=0.05, aspect=30)
#cbar = fig.colorbar(cax,format='%.3g',pad=0.05, aspect=30)
cbar.ax.set_ylabel(f'Max topographic effect [%]', rotation=270, labelpad=10)


In [ ]:
X=max_errors[['cw','cot','res']]
y=max_errors['max_effect']

regr = linear_model.LinearRegression()
regr.fit(X, y)
predicted_error = regr.predict([[5.27, 18.43,21]])
predicted_error


# Plot all data

In [ ]:
fig, axs = plt.subplots(len(data.cw.unique()), len(data.cot.unique()),sharex=True, sharey=True,layout='constrained')
fig.set_size_inches(80, 80)
#fig.tight_layout()
plt.subplots_adjust(wspace=0.05, hspace=0.05)
plt.rcParams.update({'font.size': 15})

axs[0,0].set_yscale('log')
axs[0,0].set_xscale('log')
axs[0,0].invert_yaxis()
axs[0,0].set_xlim([0.6, 15])
axs[0,0].xaxis.set_minor_locator(ticker.FixedLocator(locs=[0.7,0.8,0.9,2,3,4,5,6,7,8,9,11,12,13,14,15]))
axs[0,0].xaxis.set_minor_formatter(ticker.FixedFormatter(['0.7','','','2','','','5','','','','','','','','','15']))
axs[0,0].xaxis.set_major_locator(ticker.FixedLocator(locs=[1,10]))
axs[0,0].xaxis.set_major_formatter(ticker.FixedFormatter(["1","10"]))
axs[0,0].yaxis.set_major_locator(ticker.FixedLocator(locs=[0.1,1,10]))
axs[0,0].yaxis.set_major_formatter(ticker.FixedFormatter(["0.1","1","10"]))

res=resistivity_contrasts
var=resistivity_contrasts

rc_log= np.array(resistivity_contrasts_log)
cMin=min(resistivity_contrasts)
cMax=max(resistivity_contrasts)
colour = list(iter(plt.cm.turbo((rc_log-min(rc_log))/(max(rc_log)-min(rc_log)))))

colour_zip=zip(var, colour)
print(cMin,cMax)

colour_old = plt.cm.jet(np.linspace(0, 1, len(res)))
re_co=zip(res,colour)
cws=[]
cots=[]
#r2_stats=[]
for i,cw in enumerate(sorted(data_n1.cw.unique())):
    for j,cot in enumerate(sorted(data_n1.cot.unique())):
        data_filter=(data_n1['cot']==cot) & (data_n1['cw']==cw) & (data_n1['array'] == 'dd')
        filtered_data=data_n1[data_filter]
        
        axs[i,j].grid(which='major')
        axs[i,j].grid(which='minor',lw=0.2)
        
        #Plot line data
        for k in range(len(res)):
            #filtered_cw=filtered_cw.groupby('doi').mean().reset_index()
            x=filtered_data[f'E_{res[k]}']#filtered[f'E_{v}']
            y=filtered_data['doi']
            spl = CubicSpline(y, x)
            y_new=np.logspace(log10(y.min()),log10(y.max()),num=1001)
            axs[i,j].plot(spl(y_new),y_new,color=colour[k],zorder=1)
            axs[i,j].scatter(x=x,y=y,s=0.5,c='0',alpha=0.3,zorder=2)

            # Calculate R2 stat for each curve
            #ss_res = np.sum((x - spl(y)) ** 2)
            #ss_tot = np.sum((x - np.mean(x)) ** 2)
            #r2_manual = 1 - (ss_res / ss_tot)
            #r2_stats.append(r2_manual)

        #Plot scatter data
        #ax_plot=ax.scatter(x=filtered[f'E_{res}'],y=filtered['doi'],s=0.1,c=filtered[variable[i]],cmap='viridis')
        #ax_plot=axs[i,j].scatter(x=filtered_data[f'E_{v}'],y=filtered_data['doi'],s=0.1,c='0',alpha=0.3,zorder=2)
        
        #for k in range(len(res)): 
            #axs[i,j].scatter(x=filtered_data[f'E_{res[k]}'],y=filtered_data['doi'],s=1.5,label=res[k],color=colour_old[k])
        

        axs[i,j].set(xlabel="Topographic effect - E (Ω/Ω)", ylabel = 'Effective depth (m/m)')

        if i==15:
            axs[i,j].text(0.5, -0.15, f'{cot}', size=30,transform=axs[i,j].transAxes, weight='bold',ha='center', va='top')
        if j==0:
            axs[i,j].text(-0.12, 0.5, f'{cw}', size=30,transform=axs[i,j].transAxes,rotation=90, weight='bold',ha='center', va='center')

for ax in axs.flat:
    ax.label_outer()

fig.suptitle('Dipole-Dipole results n=1',fontsize=60,weight='bold')
fig.supylabel('Crest Width (m/m)',fontsize=60)
fig.supxlabel('Slope angle (cot(Θ))',fontsize=60)

#plot colour bar
ticks=[0.01,0.02,0.05,0.1,0.2,0.5,1,2,5,10,20,50,100] # resistivitiy contrasts
norm=mpl.colors.LogNorm(vmin=cMin, vmax=cMax)
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='turbo'),ax=axs,ticks=ticks,format='%.3g',pad=0.01, aspect=40,shrink=0.3)
cbar.ax.set_ylabel('Resistivity Contrast (Ωm/Ωm)', rotation=270, labelpad=10)


plt.savefig('Figures/resistivity contrast variations dipole dipole.png',format='png',pad_inches=0.1,bbox_inches='tight')

#print(f'R2 stat: {np.mean(r2_stats)}')


In [ ]:
fig, axs = plt.subplots(len(data.cw.unique()), len(data.cot.unique()),sharex=True, sharey=True,layout='constrained')
fig.set_size_inches(80, 80)
#fig.tight_layout()
plt.subplots_adjust(wspace=0.05, hspace=0.05)
plt.rcParams.update({'font.size': 15})

axs[0,0].set_yscale('log')
axs[0,0].set_xscale('log')
axs[0,0].invert_yaxis()
axs[0,0].set_xlim([0.6, 15])
axs[0,0].xaxis.set_minor_locator(ticker.FixedLocator(locs=[0.7,0.8,0.9,2,3,4,5,6,7,8,9,11,12,13,14,15]))
axs[0,0].xaxis.set_minor_formatter(ticker.FixedFormatter(['0.7','','','2','','','5','','','','','','','','','15']))
axs[0,0].xaxis.set_major_locator(ticker.FixedLocator(locs=[1,10]))
axs[0,0].xaxis.set_major_formatter(ticker.FixedFormatter(["1","10"]))
axs[0,0].yaxis.set_major_locator(ticker.FixedLocator(locs=[0.1,1,10]))
axs[0,0].yaxis.set_major_formatter(ticker.FixedFormatter(["0.1","1","10"]))

res=resistivity_contrasts
var=resistivity_contrasts

rc_log= np.array(resistivity_contrasts_log)
cMin=min(resistivity_contrasts)
cMax=max(resistivity_contrasts)
colour = list(iter(plt.cm.turbo((rc_log-min(rc_log))/(max(rc_log)-min(rc_log)))))

colour_zip=zip(var, colour)
print(cMin,cMax)

colour_old = plt.cm.jet(np.linspace(0, 1, len(res)))
re_co=zip(res,colour)
cws=[]
cots=[]
#r2_stats=[]
for i,cw in enumerate(sorted(data_n1.cw.unique())):
    for j,cot in enumerate(sorted(data_n1.cot.unique())):
        data_filter=(data_n1['cot']==cot) & (data_n1['cw']==cw) & (data_n1['array'] == 'w')
        filtered_data=data_n1[data_filter]
        
        axs[i,j].grid(which='major')
        axs[i,j].grid(which='minor',lw=0.2)
        
        #Plot line data
        for k in range(len(res)):
            #filtered_cw=filtered_cw.groupby('doi').mean().reset_index()
            x=filtered_data[f'E_{res[k]}']#filtered[f'E_{v}']
            y=filtered_data['doi']
            spl = CubicSpline(y, x)
            y_new=np.logspace(log10(y.min()),log10(y.max()),num=1001)
            axs[i,j].plot(spl(y_new),y_new,color=colour[k],zorder=1)
            axs[i,j].scatter(x=x,y=y,s=0.5,c='0',alpha=0.3,zorder=2)

            # Calculate R2 stat for each curve
            #ss_res = np.sum((x - spl(y)) ** 2)
            #ss_tot = np.sum((x - np.mean(x)) ** 2)
            #r2_manual = 1 - (ss_res / ss_tot)
            #r2_stats.append(r2_manual)

        #Plot scatter data
        #ax_plot=ax.scatter(x=filtered[f'E_{res}'],y=filtered['doi'],s=0.1,c=filtered[variable[i]],cmap='viridis')
        #ax_plot=axs[i,j].scatter(x=filtered_data[f'E_{v}'],y=filtered_data['doi'],s=0.1,c='0',alpha=0.3,zorder=2)
        
        #for k in range(len(res)): 
            #axs[i,j].scatter(x=filtered_data[f'E_{res[k]}'],y=filtered_data['doi'],s=1.5,label=res[k],color=colour_old[k])
        

        axs[i,j].set(xlabel="Topographic effect - E (Ω/Ω)", ylabel = 'Effective depth (m/m)')

        if i==15:
            axs[i,j].text(0.5, -0.15, f'{cot}', size=30,transform=axs[i,j].transAxes, weight='bold',ha='center', va='top')
        if j==0:
            axs[i,j].text(-0.12, 0.5, f'{cw}', size=30,transform=axs[i,j].transAxes,rotation=90, weight='bold',ha='center', va='center')

for ax in axs.flat:
    ax.label_outer()

fig.suptitle('Wenner results n=1',fontsize=60,weight='bold')
fig.supylabel('Crest Width (m/m)',fontsize=60)
fig.supxlabel('Slope angle (cot(Θ))',fontsize=60)

#plot colour bar
ticks=[0.01,0.02,0.05,0.1,0.2,0.5,1,2,5,10,20,50,100] # resistivitiy contrasts
norm=mpl.colors.LogNorm(vmin=cMin, vmax=cMax)
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='turbo'),ax=axs,ticks=ticks,format='%.3g',pad=0.01, aspect=40,shrink=0.3)
cbar.ax.set_ylabel('Resistivity Contrast (Ωm/Ωm)', rotation=270, labelpad=10)


plt.savefig('Figures/resistivity contrast variations wenner.png',format='png',pad_inches=0.1,bbox_inches='tight')

#print(f'R2 stat: {np.mean(r2_stats)}')
